In [24]:
from env import api_key

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key = api_key)

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key = api_key)

In [26]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings, persist_directory = "./chroma_db")

In [27]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pages = []
for pdf in [
    "https://mehr.spd.de/custom-static-assets/documents/Regierungsprogramm.pdf",
    "https://www.politikwechsel.cdu.de/sites/www.politikwechsel.cdu.de/files/docs/politikwechsel-fuer-deutschland-wahlprogramm-von-cdu-csu-1.pdf",
    "https://cms.gruene.de/uploads/assets/20241216_BTW25_Programmentwurf_DINA4_digital.pdf",
    "https://www.fdp.de/sites/default/files/2024-12/fdp-wahlprogramm_2025.pdf",
    "Leitantrag-Bundestagswahlprogramm-2025.pdf",
    "https://www.die-linke.de/fileadmin/user_upload/Wahlprogramm_Langfassung_Linke-BTW25_01.pdf",
    "https://bsw-vg.de/wp-content/themes/bsw/assets/downloads/BSW%20Wahlprogramm%202025.pdf"
]:  
    print(pdf)
    loader = PyPDFLoader(pdf)
    async for page in loader.alazy_load():
        pages.append(page)

https://mehr.spd.de/custom-static-assets/documents/Regierungsprogramm.pdf
https://www.politikwechsel.cdu.de/sites/www.politikwechsel.cdu.de/files/docs/politikwechsel-fuer-deutschland-wahlprogramm-von-cdu-csu-1.pdf
https://cms.gruene.de/uploads/assets/20241216_BTW25_Programmentwurf_DINA4_digital.pdf
https://www.fdp.de/sites/default/files/2024-12/fdp-wahlprogramm_2025.pdf
Leitantrag-Bundestagswahlprogramm-2025.pdf
https://www.die-linke.de/fileadmin/user_upload/Wahlprogramm_Langfassung_Linke-BTW25_01.pdf
https://bsw-vg.de/wp-content/themes/bsw/assets/downloads/BSW%20Wahlprogramm%202025.pdf


In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
all_splits = text_splitter.split_documents(pages)
_ = vector_store.add_documents(documents=all_splits)

In [32]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/root/projects/bundesbot/backend/venv/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [34]:
result = graph.invoke({"question": "Kannst du die Positionen zu Krankenkassen von Linken und AfD vergleichen?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: [Document(id='45d97e85-ea90-41d7-9436-cc1558d62131', metadata={'page': 28, 'page_label': '29', 'source': 'https://mehr.spd.de/custom-static-assets/documents/Regierungsprogramm.pdf'}, page_content='29\nMehr für Dich. Besser für Deutschland.\nRegierungsprogramm der SPD für die Bundestagswahl 2025\nDer Finanzausgleich zwischen den Krankenkassen soll gerechter ausgestaltet werden, und auch \ndie privaten Versicherungen sollen zum Risikostrukturausgleich beitragen. Ein solidarisches'), Document(id='80dd2bbe-80a3-43a4-89aa-d4887fcff230', metadata={'page': 32, 'page_label': '34', 'source': 'https://www.fdp.de/sites/default/files/2024-12/fdp-wahlprogramm_2025.pdf'}, page_content='Krankenversicherung. Daher lehnen wir eine Einheitskasse (sog. Bürgersversicherung) ab. In beiden \nVersicherungssystemen wollen wir Wechsel- und Wahlfreiheit der Versicherten stärken. Um die \nungebremste Leistungsausgabenentwicklung in der GKV in den Griff zu bekommen, sollen in Zukunft'), Document(id='eec6